In [1]:
import gzip
import random
from collections import defaultdict
import numpy as np

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [2]:
data=list(readGz("assignment1/train.json.gz"))
train_data = data[:100000]
valid_data = data[100000:200000]

In [3]:
def predict(alpha,beta_u,beta_i,m,n,gamma_u,gamma_i,user_dict,item_dict):
    rating = alpha  + (beta_u[m] if m in beta_u else 0)  + (beta_i[n] if n in beta_i else 0)
    if m in user_dict and n in item_dict:
        rating +=np.inner(gamma_u[user_dict[m]],gamma_i[item_dict[n]]) 
    return rating

In [4]:
# get initial alpha beta_i beta_u  gamma_u gamma_i
#Method to Train The Latent Factor Model. 
def latentfactor(lam,data,k,times):
    users = defaultdict(lambda: defaultdict(int))
    items = defaultdict(lambda: defaultdict(int))
    beta_u = defaultdict(float)
    beta_i = defaultdict(float)
    user_dict = defaultdict(int)
    user_num=0
    item_dict = defaultdict(int)
    item_num=0
    for i in data:
        user, item, rating = i['reviewerID'], i['itemID'], i['rating']
        users[user][item] = rating
        items[item][user] = rating
        if user not in user_dict:
            user_dict[user]=user_num
            user_num+=1
        if item not in item_dict:
            item_dict[item]=item_num
            item_num+=1
    
# generating gamma_u,gamma_i
    gamma_u=np.random.normal(scale=1./k,size=(len(users),k))
    gamma_i=np.random.normal(scale=1./k,size=(len(items),k ))

    #update alpha beta_u beta_i gamma_u gamma_i
    alpha=0
    for time in range(times):
        alpha=0
        for i in users:
            for j in users[i]:
                alpha += users[i][j] - beta_u[i] -beta_i[j] - np.inner(gamma_u[user_dict[i]],gamma_i[item_dict[j]])
        alpha /= len(data)
        
        for i in users:
            beta_u[i] = 0
            for j in users[i]:
                beta_u[i] += users[i][j]  - alpha - beta_i[j] - np.inner(gamma_u[user_dict[i]],gamma_i[item_dict[j]])
            beta_u[i] /= (lam + len(users[i])) 
            
        for j in items:
            beta_i[j] = 0
            for i in items[j]:
                beta_i[j] += items[j][i]  -alpha - beta_u[i] - np.inner(gamma_u[user_dict[i]],gamma_i[item_dict[j]])
            beta_i[j] /= (lam + len(items[j]))
    
        for i in users:
            for d in range(k):
                gamma_u[user_dict[i]][d] = 0
                for j in users[i]:
                    gamma_u[user_dict[i]][d] += gamma_i[item_dict[j]][d]*(users[i][j]  - alpha - beta_i[j]  +gamma_i[item_dict[j]][d]*gamma_i[item_dict[j]][d]-np.inner(gamma_u[user_dict[i]],gamma_i[item_dict[j]]) )
                    gamma_u[user_dict[i]][d]  /= (lam + gamma_i[item_dict[j]][d]*gamma_i[item_dict[j]][d])
                    
        for j in items:
            for d in range(k):
                gamma_i[item_dict[j]][d] = 0
                for i in items[j]:
                    gamma_i[item_dict[j]][d] += gamma_u[user_dict[i]][d]*(users[i][j]  - alpha - beta_u[i] - np.inner(gamma_u[user_dict[i]],gamma_i[item_dict[j]]) +gamma_u[user_dict[i]][d]*gamma_u[user_dict[i]][d] )
                    gamma_i[item_dict[j]][d] /= (lam + gamma_u[user_dict[i]][d]*gamma_u[user_dict[i]][d])
    return alpha,beta_u,beta_i,user_dict,item_dict,gamma_i,gamma_u

In [27]:
alpha,beta_u,beta_i,user_dict,item_dict,gamma_i,gamma_u=latentfactor(6.4,data,9,1)

In [28]:
## output prediction file

predictions = open("assignment1/predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)  
        continue
    u,i = l.strip().split('-')   
    rating=predict(alpha,beta_u,beta_i,u,i,gamma_u,gamma_i,user_dict,item_dict)
    predictions.write(u + '-' + i + ","+str(rating)+"\n")
predictions.close()